In [1]:
#Making necesarry imports
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os, sys
import re
import seaborn as sns

In [3]:
df = pd.read_json('data/reviews_Digital_Music_5.json', lines=True)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,5555991584,"[3, 3]",5,"It's hard to believe ""Memory of Trees"" came ou...","09 12, 2006",A3EBHHCZO6V2A4,"Amaranth ""music fan""",Enya's last great album,1158019200
1,5555991584,"[0, 0]",5,"A clasically-styled and introverted album, Mem...","06 3, 2001",AZPWAXJG9OJXV,bethtexas,Enya at her most elegant,991526400
2,5555991584,"[2, 2]",5,I never thought Enya would reach the sublime h...,"07 14, 2003",A38IRL0X2T4DPF,bob turnley,The best so far,1058140800
3,5555991584,"[1, 1]",5,This is the third review of an irish album I w...,"05 3, 2000",A22IK3I6U76GX0,Calle,Ireland produces good music.,957312000
4,5555991584,"[1, 1]",4,"Enya, despite being a successful recording art...","01 17, 2008",A1AISPOIIHTHXX,"Cloud ""...""",4.5; music to dream to,1200528000


In [4]:
df.shape

(64706, 9)

In [8]:
asin = ratings.asin.nunique()
asin

3568

In [9]:
reviewerID = ratings.reviewerID.nunique()
reviewerID

5541

In [10]:
sparsity = 1.0-len(ratings)/float(asin*reviewerID)
print('Sparsity : ', str(sparsity*100), '%')

Sparsity :  99.67271088817725 %


In [11]:
ratings.overall.unique()

array([5, 4, 3, 1, 2])

In [12]:
ratings_explicit = ratings[ratings.overall != 0]
ratings_implicit = ratings[ratings.overall == 0]

In [14]:
print(ratings_explicit.shape)
print(ratings_implicit.shape)

(64706, 9)
(0, 9)


In [22]:
ratings_count = pd.DataFrame(ratings_explicit.groupby(['asin'])['overall'].mean())
top10 = ratings_count.sort_values('overall', ascending = False).head(10)
print("Following Popular Music are Recommended")
top10

Following Popular Music are Recommended


,overall
asin,
B000001FKJ,5.0
B000002W5F,5.0
B00000337Z,5.0
B00000JHCL,5.0
B001HDYKQ4,5.0
B00000335M,5.0
B0007US7OO,5.0
B001NCHCOQ,5.0
B00000335I,5.0


In [23]:
ratings_matrix = ratings_explicit.pivot(index='asin', columns='reviewerID', values='overall')
userID = ratings_matrix.index
ISBN = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix.head()

(3568, 5541)


reviewerID,A08161909WK3HU7UYTMW,A1020L7BWW9RAX,A10323WWTFPSGP,A103KNDW8GN92L,A103W7ZPKGOCC9,A105188E1HFWRX,A105S56ODHGJEK,A1065Q31DNPRAN,A106BECWWKQIC9,A107R79K63CE30,...,AZSN1TO0JI87B,AZT9OGZODW8VW,AZTVHB938DIG5,AZTZ61W23JET0,AZVA13Q19KURR,AZVH70JMJ2IHX,AZVHFS4KQBQTK,AZWCAUCNLGL4H,AZY3MDFJJFI91,AZYZA8FDL48GD
asin,,,,,,,,,,,,,,,,,,,,,
5555991584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0000000ZW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000016T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000016W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000017R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#since NaNs cannot be handled by training algorithms, replacing these by 0, which indicates absence of ratings
#setting data type
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)
#checking first few rows
ratings_matrix.head()

reviewerID,A08161909WK3HU7UYTMW,A1020L7BWW9RAX,A10323WWTFPSGP,A103KNDW8GN92L,A103W7ZPKGOCC9,A105188E1HFWRX,A105S56ODHGJEK,A1065Q31DNPRAN,A106BECWWKQIC9,A107R79K63CE30,...,AZSN1TO0JI87B,AZT9OGZODW8VW,AZTVHB938DIG5,AZTZ61W23JET0,AZVA13Q19KURR,AZVH70JMJ2IHX,AZVHFS4KQBQTK,AZWCAUCNLGL4H,AZY3MDFJJFI91,AZYZA8FDL48GD
asin,,,,,,,,,,,,,,,,,,,,,
5555991584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0000000ZW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00000016T,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00000016W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00000017R,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
ratings_matrix = ratings_matrix.T

In [33]:
#rechecking the sparsity
sparsity=1.0-len(ratings_explicit)/float(asin*reviewerID)
print('The sparsity level of Music dataset is ' ,  str(sparsity*100) , ' %')

The sparsity level of Music dataset is  99.67271088817725  %


In [34]:
#setting global variables
global metric,k
k=10
metric='cosine'

In [35]:
#This function finds k similar users given the user_id and ratings matrix 
#These similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)
    loc = ratings.index.get_loc(user_id)
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
            
    return similarities,indices

In [39]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = metric, k=k):
    prediction=0
    user_loc = ratings.index.get_loc(user_id)
    item_loc = ratings.columns.get_loc(item_id)
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.iloc[user_loc,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_loc:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_loc]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >5:
        prediction = 5
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [40]:
predict_userbased('A103W7ZPKGOCC9','B00000016T',ratings_matrix);


Predicted rating for user A103W7ZPKGOCC9 -> item B00000016T: 0


In [41]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(ratings_matrix, metric='cosine')
item_similarity = pairwise_distances(ratings_matrix.T, metric='cosine')

In [42]:
user_similarity

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [43]:
item_similarity

array([[0.        , 1.        , 0.93461197, ..., 1.        , 0.94490923,
        1.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.93461197, 1.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 0.87431313,
        1.        ],
       [0.94490923, 1.        , 1.        , ..., 0.87431313, 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.        ]])

In [44]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T   
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [46]:
user_prediction = predict(ratings_matrix, user_similarity, type='user')
item_prediction = predict(ratings_matrix, item_similarity, type='item')

ValueError: Unable to coerce to DataFrame, shape must be (5541, 3568): given (5541, 1)